In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/sales-analysis'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sales-analysis/SalesKaggle3.csv


In [3]:
#import libraries 

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [4]:
# 1. Read the dataset
df = pd.read_csv("/kaggle/input/sales-analysis/SalesKaggle3.csv")

In [5]:
# 2. Initial Examination of the Data
print("Initial Data Shape:", df.shape)
print("Columns:", df.columns.tolist())
print(df.head())

Initial Data Shape: (198917, 14)
Columns: ['Order', 'File_Type', 'SKU_number', 'SoldFlag', 'SoldCount', 'MarketingType', 'ReleaseNumber', 'New_Release_Flag', 'StrengthFactor', 'PriceReg', 'ReleaseYear', 'ItemCount', 'LowUserPrice', 'LowNetPrice']
   Order   File_Type  SKU_number  SoldFlag  SoldCount MarketingType  \
0      2  Historical     1737127       0.0        0.0             D   
1      3  Historical     3255963       0.0        0.0             D   
2      4  Historical      612701       0.0        0.0             D   
3      6  Historical      115883       1.0        1.0             D   
4      7  Historical      863939       1.0        1.0             D   

   ReleaseNumber  New_Release_Flag  StrengthFactor  PriceReg  ReleaseYear  \
0             15                 1        682743.0     44.99         2015   
1              7                 1       1016014.0     24.81         2005   
2              0                 0        340464.0     46.00         2013   
3              4  

In [6]:
# 3. Drop Irrelevant Columns
# The "Order" column is a sequential counter which we can drop.
df.drop(columns=["Order"], inplace=True)
sku_ids = df["SKU_number"].copy()


In [7]:
print(df.head())

    File_Type  SKU_number  SoldFlag  SoldCount MarketingType  ReleaseNumber  \
0  Historical     1737127       0.0        0.0             D             15   
1  Historical     3255963       0.0        0.0             D              7   
2  Historical      612701       0.0        0.0             D              0   
3  Historical      115883       1.0        1.0             D              4   
4  Historical      863939       1.0        1.0             D              2   

   New_Release_Flag  StrengthFactor  PriceReg  ReleaseYear  ItemCount  \
0                 1        682743.0     44.99         2015          8   
1                 1       1016014.0     24.81         2005         39   
2                 0        340464.0     46.00         2013         34   
3                 1        334011.0    100.00         2006         20   
4                 1       1287938.0    121.95         2010         28   

   LowUserPrice  LowNetPrice  
0         28.97        31.84  
1          0.00        1

In [8]:
df.head()

,File_Type,SKU_number,SoldFlag,SoldCount,MarketingType,ReleaseNumber,New_Release_Flag,StrengthFactor,PriceReg,ReleaseYear,ItemCount,LowUserPrice,LowNetPrice
0,Historical,1737127,0.0,0.0,D,15,1,682743.0,44.99,2015,8,28.97,31.84
1,Historical,3255963,0.0,0.0,D,7,1,1016014.0,24.81,2005,39,0.00,15.54
2,Historical,612701,0.0,0.0,D,0,0,340464.0,46.00,2013,34,30.19,27.97
3,Historical,115883,1.0,1.0,D,4,1,334011.0,100.00,2006,20,133.93,83.15
4,Historical,863939,1.0,1.0,D,2,1,1287938.0,121.95,2010,28,4.00,23.99


In [10]:
# 4. Handle Missing Values
# Let’s check for missing values.
print("Missing values per column:\n", df.isnull().sum())

Missing values per column:
 File_Type                0
SKU_number               0
SoldFlag            122921
SoldCount           122921
MarketingType            0
ReleaseNumber            0
New_Release_Flag         0
StrengthFactor           0
PriceReg                 0
ReleaseYear              0
ItemCount                0
LowUserPrice             0
LowNetPrice              0
dtype: int64


In [11]:
numeric_features = ["SoldCount", "ReleaseNumber", "StrengthFactor", "PriceReg"]
categorical_features = ["File_Type", "MarketingType", "New_Release_Flag"]

In [12]:
# 5. Create Preprocessing Pipelines

# Numeric pipeline: imputation then scaling.
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Categorical pipeline: imputation and one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# Combine pipelines using ColumnTransformer.
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ])

In [14]:
# Check how many missing values are in the SoldFlag column
print("Missing values in SoldFlag before filling:", df["SoldFlag"].isnull().sum())

# Fill missing values in SoldFlag with 0 (or another appropriate default)
df["SoldFlag"] = df["SoldFlag"].fillna(0)

# Now safely cast to int
y = df["SoldFlag"].astype(int)
X = df.drop(columns=["SoldFlag", "SKU_number"])

Missing values in SoldFlag before filling: 122921


In [15]:
X = df.drop(columns=["SoldFlag", "SKU_number"])  # Exclude target and identifier if needed
y = df["SoldFlag"].astype(int) 

In [16]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

# Create a full pipeline with the preprocessing steps and the XGBoost classifier.
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", XGBClassifier(
        random_state=42, 
        use_label_encoder=False, 
        eval_metric="logloss"
    ))
])

# Optionally, you can print out the pipeline summary to verify its structure.
print(model)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SoldCount', 'ReleaseNumber',
                                                   'StrengthFactor',
                                                   'PriceReg']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                             

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix


In [21]:
# 8. Split Data into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)


Train shape: (159133, 11) Test shape: (39784, 11)


In [22]:
# 9. Fit the Pipeline on Training Data using XGBoost inside the model pipeline
model.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SoldCount', 'ReleaseNumber',
                                                   'StrengthFactor',
                                                   'PriceReg']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

In [23]:
# 10. (Optional) Transform Data for Model Input outside the pipeline if needed
# Note: Since XGBoost is encapsulated in the pipeline, you don't need to call preprocessor separately
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)
print("Preprocessed training data shape:", X_train_preprocessed.shape)

Preprocessed training data shape: (159133, 10)


In [24]:
# 11. Make Predictions and Evaluate using XGBoost
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Obtain probabilities for the positive class

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 1.0
ROC AUC Score: 0.9999999999999999
Confusion Matrix:
 [[37185     0]
 [    0  2599]]


In [25]:
# 12. Apply the Preprocessing and Model Pipeline to the Entire Data for Scoring
# This is useful if you want to score all products (e.g., active inventory)
# Since the model includes the preprocessor, directly use model.predict_proba on X:
predictions = model.predict_proba(X)[:, 1]

# Save or attach predictions to your DataFrame
df["Sale_Probability"] = predictions


In [26]:
# Optionally, if you saved SKU_number separately and need to attach it:
df["SKU_number"] = sku_ids  # sku_ids should have been saved earlier


In [27]:
# Export the scored DataFrame to a CSV file
df.to_csv("scored_inventory.csv", index=False)
print("Scored file saved as 'scored_inventory.csv'")

Scored file saved as 'scored_inventory.csv'


In [28]:
import os
for dirname, _, filenames in os.walk('/kaggle/working/scored_inventory.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [29]:
# 1. Read the dataset
df = pd.read_csv("/kaggle/working/scored_inventory.csv")

In [31]:
print("Initial Data Shape:", df.shape)
print(df.head())

Initial Data Shape: (198917, 14)
    File_Type  SKU_number  SoldFlag  SoldCount MarketingType  ReleaseNumber  \
0  Historical     1737127       0.0        0.0             D             15   
1  Historical     3255963       0.0        0.0             D              7   
2  Historical      612701       0.0        0.0             D              0   
3  Historical      115883       1.0        1.0             D              4   
4  Historical      863939       1.0        1.0             D              2   

   New_Release_Flag  StrengthFactor  PriceReg  ReleaseYear  ItemCount  \
0                 1        682743.0     44.99         2015          8   
1                 1       1016014.0     24.81         2005         39   
2                 0        340464.0     46.00         2013         34   
3                 1        334011.0    100.00         2006         20   
4                 1       1287938.0    121.95         2010         28   

   LowUserPrice  LowNetPrice  Sale_Probability  
0   

In [33]:
df.head()

,File_Type,SKU_number,SoldFlag,SoldCount,MarketingType,ReleaseNumber,New_Release_Flag,StrengthFactor,PriceReg,ReleaseYear,ItemCount,LowUserPrice,LowNetPrice,Sale_Probability
0,Historical,1737127,0.0,0.0,D,15,1,682743.0,44.99,2015,8,28.97,31.84,0.000007
1,Historical,3255963,0.0,0.0,D,7,1,1016014.0,24.81,2005,39,0.00,15.54,0.000007
2,Historical,612701,0.0,0.0,D,0,0,340464.0,46.00,2013,34,30.19,27.97,0.000009
3,Historical,115883,1.0,1.0,D,4,1,334011.0,100.00,2006,20,133.93,83.15,0.999938
4,Historical,863939,1.0,1.0,D,2,1,1287938.0,121.95,2010,28,4.00,23.99,0.999917
